In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *

In [2]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

['Move', 'Move', 'Move', 'Shoot', 'Shoot', 'Shoot']
['Move', 'Move', 'Shoot', 'Move', 'Shoot', 'Shoot']
['Move', 'Move', 'Shoot', 'Shoot', 'Move', 'Shoot']
['Move', 'Shoot', 'Move', 'Move', 'Shoot', 'Shoot']
['Move', 'Shoot', 'Move', 'Shoot', 'Move', 'Shoot']


In [3]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))

In [4]:
gb = GridBuilder()

gb.place( WaterTile(),      (0,4) )
gb.place( WaterTile(),      (0,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,7) )

gb.place( Destructable(CorporateTile(), GroundTile(), 1), (1,1) )
gb.place( ForestTile(),     (1,2) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,3) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,4) )
gb.place( WaterTile(),      (1,5) )
gb.place( WaterTile(),      (1,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (1,7) )

gb.place( ForestTile(),     (2,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,6) )
gb.place( WaterTile(),      (2,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (3,1) )
gb.place( WaterTile(),      (3,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,1) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,5) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,6) )

gb.place( ForestFireTile(), (5,1) )
gb.place( ForestFireTile(), (5,2) )
gb.place( SpawnTile(),      (5,4) )
gb.place( ForestTile(),     (5,7) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (6,0) )
gb.place( SpawnTile(),      (6,2) )
gb.place( ForestFireTile(), (6,3) )
gb.place( ForestFireTile(), (6,4) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,0) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,1) )
gb.place( ForestTile(),     (7,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,7) )

gb.place_on_tile( Mech("Siege Mech", 5, 4),     (3,2) )
gb.place_on_tile( Mech("Artillery Mech", 5, 1), (5,3) )
gb.place_on_tile( Mech("Boulder Mech", 5, 5, 4),   (7,3) )
gb.place_on_tile( Vek("Psy", 5, 2),             (4,2) )
gb.place_on_tile( Vek("Alpha Firefly", 6, 5, 2),   (4,4) )
gb.place_on_tile( Vek("Scarab", 3, 3),          (6,1) )
gb.place_on_tile( Vek("Firefly", 4, 4, 2),         (6,5) )
gb.place_on_tile( Unit("Boulder", max_health=1, health=1, moves=0), (7,5) )

grid = gb.to_grid()

for mech in grid.mechs:
    mech.add(Move(mech,grid))
    mech.add(Repair(mech))
    
for vek in grid.veks:
    vek.add(Move(vek, grid))

    
siege_mech     = grid.units['Siege Mech']
artillery_mech = grid.units['Artillery Mech']
boulder_mech   = grid.units['Boulder Mech']

firefly        = grid.units['Firefly']
alpha_firefly  = grid.units['Alpha Firefly']
scarab         = grid.units['Scarab']
psy            = grid.units['Psy']


siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))
boulder_mech.add(Artillery(boulder_mech, grid, BoulderShell, 2))
artillery_mech.add(Artillery(artillery_mech, grid, RegularShell, 1))

firefly.add(Beam(firefly, grid, VekBeam, 1))
alpha_firefly.add(Beam(alpha_firefly, grid, VekBeam, 3))
scarab.add(Artillery(scarab, grid, VekShell, 1))

firefly.target = CommandDecorator(firefly, VekBeam(firefly, grid, Compass.NORTH, 1))
alpha_firefly.target = CommandDecorator(alpha_firefly, VekBeam(alpha_firefly, grid, Compass.NORTH, 3))
scarab.target = CommandDecorator(scarab, VekShell(scarab, grid, 1, (1,1)))
    
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [5]:
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [6]:
grid.get_tile((6,1)).visitor.health -= 1

In [7]:
sc = SpawnCommand(grid, (6,2), Vek("Scarab"))

In [8]:
sc.execute()

In [9]:
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡,Scarab ♡♡♡,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [10]:
# class DFS:
#     @staticmethod
#     def rate(grid):
#         power, mech_alive, mech_total, vek_alive, vek_total = self.rate_base(grid)
#         return {'Power':power, 'Veks':vek_alive, 'Vek Total Health':vek_total, 'Mechs':mech_alive, 'Mech Total Health':mech_total}
    
#     @staticmethod
#     def rate_base(grid):
#         commands = []
#         for vek in grid.veks:
#             commands.append(vek.target)
            
#         commands.extend([
#             DamageUnitCommand(alpha_firefly, 1), 
#             DamageCommand(grid, (5,1), 1),
#             DamageCommand(grid, (5,1), 2),
#             DamageCommand(grid, (6,3), 1),
#             DamageCommand(grid, (6,4), 1),
#             DamageCommand(grid, (6,2), 1),
#             DamageCommand(grid, (5,4), 1),
#         ])
            
#         for command in commands:
#             command.execute()
        
#         power = sum(sum(tile.type_object.health for tile in tiles if '🏢' in repr(tile) or '🏘️' in repr(tile)) for tiles in grid.tiles)
#         mech_alive = sum(1 for mech in grid.mechs if mech.health > 0)
#         mech_total = sum(mech.health for mech in grid.mechs)
#         vek_alive = sum(1 for vek in grid.veks if vek.health > 0)
#         vek_total = sum(vek.health for vek in grid.veks)
        
#         for command in commands[::-1]:
#             command.undo()
            
#         return power, mech_alive, mech_total, vek_alive, vek_total
    
#     @staticmethod
#     def rate_num(grid):
#         power, mech_alive, mech_total, vek_alive, vek_total = self.rate_base(grid)
#         return 0 + power*5 - vek_total - vek_alive*10 + mech_total + mech_alive*10
    
#     def alter_rate_and_unalter(self, action):
#         action.execute()
#         score = self.rate_num(self.grid)
#         action.undo()
#         return score
    
#     def __init__(self, grid):
#         self.grid = grid
#         self.ex = Executor()
#         self.explored = {}
#         self.current = ""
#         self.search()
        
#     def order(self, action):
#         rating_before = self.rate(grid)
#         action.execute()
#         rating_after = self.rate(grid)
#         action.undo()
#         return self.get_score(rating_before, rating_after)
        
#     @staticmethod
#     def get_score(rating_before, rating_after):
#         score = 0
        
#         score += (rating_after['Power'] - rating_before['Power'])*5
#         score += rating_before['Vek Total Health'] - rating_after['Vek Total Health']
#         score += (rating_before['Veks'] - rating_after['Veks'])*10
#         score += (rating_after['Mechs'] - rating_before['Mechs'])*10
#         score += rating_after['Mech Total Health'] - rating_before['Mech Total Health']
        
#         return score
    
#     @staticmethod
#     def get_score_single(rating):
#         score = 0
#         score += rating['Power']*5
#         score -= rating['Vek Total Health']
#         score -= rating['Veks']*10
#         score += rating['Mechs']*10
#         score += rating['Mech Total Health']
#         return score
        
#     @staticmethod
#     def gen_rated_frontier(grid):
#         """returns action and rating"""
#         frontier = []
#         for mech in grid.mechs:
#             for k,action in mech.gen_actions().items():
#                 score = self.alter_rate_and_unalter(action)
#                 if score 
                
#                 frontier.append((action, score))
#         return frontier
        
#     @staticmethod
#     def gen_frontier(grid):
#         frontier = []
#         for mech in grid.mechs:
#             for k,action in mech.gen_actions().items():
#                 frontier.append(action)
#         return frontier
    
#     def search2(self):
#         pass
    
#     # now go through all options for 1 round
#     def search(self):
#         frontier = self.gen_frontier(self.grid)
#         if not frontier:
#             score = self.rate(self.grid)
# #             print(self.current, score)
#             self.explored[self.current] = score
#             return
    
        
        
#         frontier = [x for x in frontier if isinstance(x.command, MoveCommand) or self.rate_num(x) > 0]
#         # frontier.sort(key=self.order, reverse=True)

#         for i,action in enumerate(frontier):
#             char = str(i) + '_'
#             self.current += char
#             action.execute()
#             self.search()
#             action.undo()
#             self.current = self.current[:-len(char)]
            
#     def __repr__(self):
#         string = ""
#         for i,v in self.explored.items():
#             string += f"{i}) {v}"
#         return string
    
#     def find_best(self):
#         ls = [(i,v) for i,v in self.explored.items()]
#         ls.sort(key=lambda iv: self.get_score_single(iv[1]), reverse=True)
#         return ls
        

In [11]:
class DFS:
    def __init__(self, grid):
        self.grid = grid
        self.explored = {}
        self.current = ""
        self.search(self.rate(self.grid))
        
    def search(self, curr_score):
        frontier = self.gen_rated_frontier(self.grid)
        [print(x) for x in frontier]
        print()
        frontier = list(filter(lambda x:(isinstance(x[0].command,MoveCommand) and x[1] >= curr_score) or x[1]>curr_score,frontier))
        frontier.sort(key=lambda x:x[1])
        [print(x) for x in frontier]
    
    @classmethod
    def gen_rated_frontier(cls,grid):
        """returns action and rating"""
        frontier = []
        for mech in grid.mechs:
            for k,action in mech.gen_actions().items():
                score = cls.alter_rate_and_unalter(grid, action)
                frontier.append((action, score))
                
        return frontier
    
    @classmethod
    def rate_dict(cls, grid):
        power, mech_alive, mech_total, vek_alive, vek_total = cls.rate_base(grid)
        return {'Power':power, 'Veks':vek_alive, 'Vek Total Health':vek_total, 'Mechs':mech_alive, 'Mech Total Health':mech_total}
    
    @staticmethod
    def rate_base(grid):
        commands = []
            
        commands.extend([
            DamageUnitCommand(alpha_firefly, 1), 
            DamageCommand(grid, (5,1), 1),
            DamageCommand(grid, (5,1), 2),
            DamageCommand(grid, (6,3), 1),
            DamageCommand(grid, (6,4), 1),
            DamageCommand(grid, (6,2), 1),
            DamageCommand(grid, (5,4), 1),
        ])
        
        for vek in grid.veks:
            commands.append(vek.target)
            
        for command in commands:
            command.execute()
        
        power = sum(sum(tile.type_object.health for tile in tiles if '🏢' in repr(tile) or '🏘️' in repr(tile)) for tiles in grid.tiles)
        mech_alive = sum(1 for mech in grid.mechs if mech.health > 0)
        mech_total = sum(mech.health for mech in grid.mechs)
        vek_alive = sum(1 for vek in grid.veks if vek.health > 0)
        vek_total = sum(vek.health for vek in grid.veks)
        
        for command in commands[::-1]:
            command.undo()
            
        return power, mech_alive, mech_total, vek_alive, vek_total
    
    @classmethod
    def rate(cls, grid):
        power, mech_alive, mech_total, vek_alive, vek_total = cls.rate_base(grid)
        return 0 + power*5 - vek_total - vek_alive*10 + mech_total + mech_alive*10
    
    @classmethod
    def alter_rate_and_unalter(cls, grid, action):
        action.execute()
        score = cls.rate(grid)
        action.undo()
        return score

In [12]:
DFS(grid)

(MOVE (3, 2) to (1, 2), 23)
(MOVE (3, 2) to (3, 2), 23)
(MOVE (3, 2) to (3, 3), 23)
(MOVE (3, 2) to (0, 2), 23)
(MOVE (3, 2) to (2, 1), 23)
(MOVE (3, 2) to (2, 0), 23)
(MOVE (3, 2) to (2, 3), 23)
(MOVE (3, 2) to (4, 3), 23)
(MOVE (3, 2) to (2, 2), 23)
(MOVE (3, 2) to (3, 4), 25)
(MOVE (3, 2) to (2, 4), 25)
(MOVE (3, 2) to (3, 5), 23)
(MOVE (5, 3) to (6, 4), 12)
(MOVE (5, 3) to (5, 4), 12)
(MOVE (5, 3) to (3, 3), 23)
(MOVE (5, 3) to (5, 5), 17)
(MOVE (5, 3) to (5, 6), 23)
(MOVE (5, 3) to (6, 3), 12)
(MOVE (5, 3) to (7, 4), 23)
(MOVE (5, 3) to (2, 3), 23)
(MOVE (5, 3) to (4, 3), 23)
(MOVE (5, 3) to (5, 0), 23)
(MOVE (5, 3) to (3, 4), 17)
(MOVE (5, 3) to (5, 1), 12)
(MOVE (5, 3) to (5, 2), 25)
(MOVE (5, 3) to (5, 3), 25)
(MOVE (7, 3) to (7, 3), 25)
(MOVE (7, 3) to (6, 4), 24)
(MOVE (7, 3) to (5, 4), 24)
(MOVE (7, 3) to (5, 5), 29)
(MOVE (7, 3) to (7, 4), 25)
(MOVE (7, 3) to (6, 3), 24)
(MOVE (7, 3) to (7, 2), 25)

(MOVE (3, 2) to (1, 2), 23)
(MOVE (3, 2) to (3, 2), 23)
(MOVE (3, 2) to (3,

In [13]:
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡,Scarab ♡♡♡,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [14]:
frontier = DFS.gen_rated_frontier(grid)

In [15]:
frontier.sort(key=lambda x,y:y)

TypeError: <lambda>() missing 1 required positional argument: 'y'

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    